In [ ]:
!pip install contextily
!pip install osmnx
!pip install ipyleaflet
!pip install geopandas matplotlib
!pip install folium

In [ ]:
!pip install contextily

In [ ]:
!pip install osmnx

In [ ]:
!pip install ipyleaflet

In [ ]:
!pip install geopandas matplotlib

In [ ]:
!pip install folium


In [9]:
from IPython import get_ipython
from IPython.display import display
#import contextily
#import osmnx
import ipyleaflet
import geopandas as gpd
import pandas as pd
import folium

def load_and_preprocess_data(geojson_file, pop_file, smallarea_file, dwellings_file):
    lina1 = gpd.read_file(geojson_file)
    pop = pd.read_csv(pop_file)
    smallarea = gpd.read_file(smallarea_file)
    dwellings = pd.read_csv(dwellings_file)

    # Pad 'smasvaedi' column with leading zeros
    pop['smasvaedi'] = pop['smasvaedi'].astype(str).str.zfill(4)

    # Transform to WGS84
    smallarea_wgs84 = smallarea.to_crs(epsg=4326)
    lina1_wgs84 = lina1.to_crs(epsg=4326)

    # Filter data
    pop2024 = pop[(pop['ar'] == 2024) & (pop['aldursflokkur'] == "0-4 ára") & (pop['kyn'] == 1)]
    all_dwellings = dwellings[dwellings['framvinda'] == "Fullbúið"].groupby('smasvaedi')['Fjöldi'].sum().reset_index()

    # Join dataframes
    pop2024_smallarea = smallarea_wgs84.merge(pop2024, left_on='smsv', right_on='smasvaedi', how='left')
    all_dwellings_smallarea = smallarea_wgs84.merge(all_dwellings, left_on='fid', right_on='smasvaedi', how='left')

    return lina1_wgs84, pop2024_smallarea, all_dwellings_smallarea

def create_map(lina1_wgs84, all_dwellings_smallarea):
    m = folium.Map()

    # The 'dagsetning' column does not exist in all_dwellings_smallarea
    #all_dwellings_smallarea['dagsetning'] = all_dwellings_smallarea['dagsetning'].astype(str)

    # Add polygons to the map
    folium.Choropleth(
        geo_data=all_dwellings_smallarea[all_dwellings_smallarea['nuts3'] == '001'].to_json(),
        name="choropleth",
        data=all_dwellings_smallarea,
        columns=["smasvaedi", "Fjöldi"],
        key_on="feature.properties.smasvaedi",
        fill_color="YlGn",
        fill_opacity=0.7,
        line_opacity=0.2,
        legend_name="Dwellings"
    ).add_to(m)

    # Add markers for lina1_wgs84
    for _, r in lina1_wgs84.iterrows():
        folium.Marker(location=[r.geometry.y, r.geometry.x]).add_to(m)

    # Display the map
    display(m)

def main():
  geojson_file = "cityline_2025.geojson"
  pop_file = "ibuafjoldi.csv"
  smallarea_file = "smasvaedi_2021.json"
  dwellings_file = "ibudir.csv"

  lina1_wgs84, pop2024_smallarea, all_dwellings_smallarea = load_and_preprocess_data(geojson_file, pop_file, smallarea_file, dwellings_file)
  create_map(lina1_wgs84, all_dwellings_smallarea)  # Call the function to create the map

if __name__ == "__main__":
  main()


TypeError: Object of type Timestamp is not JSON serializable